In [9]:
'''
2、哪些商品真的在打折呢？
要求：
① 针对每个商品，评估其打折的情况
② 针对在打折的商品，其折扣率是多少
   * 用bokeh绘制折线图：x轴为折扣率，y轴为商品数量
③ 按照品牌分析，不同品牌的打折力度
   * 用bokeh绘制浮动散点图，y坐标为品牌类型，x坐标为折扣力度
提示：
① 打折情况评估方法：
   真打折：商品的价格在10天内有波动、双11价格为10天内最低价、不存在涨价现象
   不打折：商品价格无变化
② 针对每个商品做price字段的value值统计，查看价格是否有波动，可以先用pd.cut将date分为不同周期：'双十一前','双十一当天','双十一后'，得到period字段
   data[['id','price','date']].groupby(['id','price']).min()
   针对统计出来的结果，如果按照id和price分组仍只有一个唯一值，则说明价格未变，没打折；否则为打折
③ 折扣率 = 双十一当天价格 / 双十一之前价格
④ 作图过程中，清除掉折扣率大于95%的数据

'''

"\n2、哪些商品真的在打折呢？\n要求：\n① 针对每个商品，评估其打折的情况\n② 针对在打折的商品，其折扣率是多少\n   * 用bokeh绘制折线图：x轴为折扣率，y轴为商品数量\n③ 按照品牌分析，不同品牌的打折力度\n   * 用bokeh绘制浮动散点图，y坐标为品牌类型，x坐标为折扣力度\n提示：\n① 打折情况评估方法：\n   真打折：商品的价格在10天内有波动、双11价格为10天内最低价、不存在涨价现象\n   不打折：商品价格无变化\n② 针对每个商品做price字段的value值统计，查看价格是否有波动，可以先用pd.cut将date分为不同周期：'双十一前','双十一当天','双十一后'，得到period字段\n   data[['id','price','date']].groupby(['id','price']).min()\n   针对统计出来的结果，如果按照id和price分组仍只有一个唯一值，则说明价格未变，没打折；否则为打折\n③ 折扣率 = 双十一当天价格 / 双十一之前价格\n④ 作图过程中，清除掉折扣率大于95%的数据\n\n"

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

import warnings
warnings.filterwarnings('ignore') 
# 不发出警告

from bokeh.io import output_notebook
output_notebook()
# 导入notebook绘图模块

from bokeh.plotting import figure,show,output_file
from bokeh.models import ColumnDataSource
# 导入图表绘制、图标展示模块
# 导入ColumnDataSource模块

Loading BokehJS ...

In [2]:
# 查看数据, 计算商品总数、品牌总数

import os
os.chdir(r'C:\Users\Administrator\Python数据\网易数据分析\1项目资料\项目08电商打折套路解析')
# 创建工作路径

df = pd.read_excel('双十一淘宝美妆数据.xlsx',sheetname=0,header=0,index_col=0)

# df_length = len(df)
# df_columns = df.columns.tolist()
df.fillna(0,inplace = True)   # 填充缺失值

df['date'] = df.index.day     # 提取销售日期

In [3]:
# 针对每个商品，评估其打折的情况
#   真打折：商品的价格在10天内有波动、双11价格为10天内最低价、不存在涨价现象
#   不打折：商品价格无变化

data2 = df[['id','title','店名','date','price']]
data2['period'] = pd.cut(data2['date'],[4,10,11,14],labels = ['双十一前','双十一当天','双十一后']) # 添加了'period'字段
# print(data2.head())
# 筛选数据

price = data2[['id','price','period']].groupby(['id','price']).min()
price.reset_index(inplace = True)
print(price[:5])
# 针对每个商品做price字段的value值统计，查看价格是否有波动

id_count = price['id'].value_counts()
id_type1 = id_count[id_count == 1].index
id_type2 = id_count[id_count != 1].index
# 筛选出“不打折”和“真打折”的商品id

n1 = len(id_type1)
n2 = len(id_type2)
print('真打折的商品数量约占比%.2f%%，不打折的商品数量约占比%.2f%%' % (n2/len(id_count)*100, n1/len(id_count)*100))

             id  price period
0  A10027317366  159.0  双十一当天
1  A10027317366  188.0   双十一后
2  A10027317366  258.0   双十一前
3  A10588608182  189.0   双十一后
4  A10588608182  249.0   双十一前
真打折的商品数量约占比24.73%，不打折的商品数量约占比75.27%


In [4]:
# 针对在打折的商品，其折扣率是多少

result3_data1 = data2[['id','price','period','店名']].groupby(['id','period']).min()
result3_data1.reset_index(inplace = True)
# 筛选数据

result3_before11 = result3_data1[result3_data1['period'] == '双十一前']
result3_at11 = result3_data1[result3_data1['period'] == '双十一当天']
result3_data2 = pd.merge(result3_at11,result3_before11,on = 'id')
# 筛选出商品双十一当天及双十一之前的价格，pd.merge()合并数据时，列名相同的，默认添加‘_x’'_y'

result3_data2['zkl'] = result3_data2['price_x'] / result3_data2['price_y']
# 计算折扣率

print(result3_data1.head(3),'\n')
print(result3_data2.head(3))

             id period  price    店名
0  A10027317366   双十一前  258.0  相宜本草
1  A10027317366  双十一当天  159.0  相宜本草
2  A10027317366   双十一后  188.0  相宜本草 

             id period_x  price_x  店名_x period_y  price_y  店名_y       zkl
0  A10027317366    双十一当天   159.00  相宜本草     双十一前    258.0  相宜本草  0.616279
1  A10588608182    双十一当天   189.00   欧莱雅     双十一前    249.0   欧莱雅  0.759036
2  A10847151685    双十一当天   125.99   佰草集     双十一前    240.0   佰草集  0.524958


In [11]:
# 用bokeh绘制折线图：x轴为折扣率，y轴为商品数量占比

from bokeh.models import HoverTool # 悬停工具
from bokeh.palettes import brewer 
from bokeh.layouts import gridplot
# 导入模块

bokeh_data = result3_data2[['id','zkl']].dropna()
bokeh_data['zkl_range'] = pd.cut(bokeh_data['zkl'],bins = np.linspace(0,1,21))
bokeh_data2 = bokeh_data.groupby('zkl_range').count().iloc[:-1] # 这里去掉折扣率在0.95-1之间的数据，该区间内数据zkl大部分为1，不打折
bokeh_data2['zkl_pre'] = bokeh_data2['zkl']/bokeh_data2['zkl'].sum()
print('bokeh_data2\n',bokeh_data2.head(3))
# 将数据按照折扣率拆分为不同区间，并统计不同1扣率的商品数量

# 创建数据
output_file('0802.html')
bokeh_data2.index = bokeh_data2.index.astype(np.str)
lst_brand = bokeh_data2.index.tolist()
# 上面两行，要在source之前
source1 = ColumnDataSource(data=bokeh_data2)

# print(type(lst_brand))

hover = HoverTool(tooltips=[('折扣率','@zkl')])  # 设置标签显示内容

p = figure(plot_width=900, plot_height=350, title="商品折扣率统计",x_range= lst_brand,
           tools=[hover,'reset,xwheel_zoom,pan,crosshair'])
# 构建绘图空间

p.line(x='zkl_range',y='zkl_pre',source = source1,     # 设置x，y值, source → 数据源
       line_width=2, line_alpha = 0.8, line_color = 'blue',line_dash = [10,4])   # 线型基本设置
# 绘制折线图
p.circle(x='zkl_range',y='zkl_pre',source = source1, size = 8,color = 'red',alpha = 0.8)

p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
# 设置其他参数

show(p)

bokeh_data2
              id  zkl   zkl_pre
zkl_range                     
(0.0, 0.05]   0    0  0.000000
(0.05, 0.1]   1    1  0.001272
(0.1, 0.15]   4    4  0.005089


In [ ]:
a3 = bokeh_data2.reset_index()
a3['zkl_range'].astype(np.str)

In [9]:
# 按照品牌分析，不同品牌的打折力度
# 用bokeh绘制浮动散点图，y坐标为品牌类型，x坐标为折扣力度

from bokeh.transform import jitter

brands = result3_data2['店名_y'].dropna().unique().tolist()
# 得到y坐标

bokeh_data = result3_data2[['id','zkl','店名_y']].dropna()
bokeh_data = bokeh_data[bokeh_data['zkl'] < 0.95]
bokeh_data.columns = ['id','zkl','PP'] # 将中文改为英文
source = ColumnDataSource(data = bokeh_data)
# 创建数据

hover = HoverTool(tooltips=[("折扣率", "@zkl"),
                           ("品牌", "@PP")])  # 设置标签显示内容

p = figure(plot_width=800, plot_height=600,y_range=brands,title="不同品牌折扣率情况",
          tools=[hover,'box_select,reset,ywheel_zoom,pan,crosshair'])

p.circle(x='zkl', 
         y=jitter('PP', width=0.6, range=p.y_range),# p.y_range 指的是上一句里的'y_range'
         source=source, alpha=0.3)
# jitter参数 → 'day'：第一参数，这里指y的值，width：间隔宽度比例，range：分类范围对象，这里和y轴的分类一致

# p.xgrid.grid_line_color = None
# p.ygrid.grid_line_color = None
# 设置其他参数

show(p)

In [14]:
'''
3、商家营销套路挖掘？
要求：
① 解析出不同品牌的参与打折商品比例及折扣力度，并做散点图，总结打折套路
   * 用bokeh绘制散点图，x轴为参与打折商品比例，y轴为折扣力度，点的大小代表该品牌参加双11活动的商品总数
提示：
① 折扣力度为该品牌所有打折商品的折扣均值，这里去掉品牌中不打折的数据
② 绘制散点图后，可以将x、y轴绘制均值辅助线，将绘图空间分为四个象限，基于该象限来总结套路

'''

'\n3、商家营销套路挖掘？\n要求：\n① 解析出不同品牌的参与打折商品比例及折扣力度，并做散点图，总结打折套路\n   * 用bokeh绘制散点图，x轴为参与打折商品比例，y轴为折扣力度，点的大小代表该品牌参加双11活动的商品总数\n提示：\n① 折扣力度为该品牌所有打折商品的折扣均值，这里去掉品牌中不打折的数据\n② 绘制散点图后，可以将x、y轴绘制均值辅助线，将绘图空间分为四个象限，基于该象限来总结套路\n\n'

In [10]:
# 解析出不同品牌的参与打折商品比例及折扣力度，并做散点图，总结打折套路

data_zk = result3_data2[result3_data2['zkl']<0.95]  # 删除未打折数据
result4_zkld = data_zk.groupby('店名_y')['zkl'].mean()
# 筛选出不同品牌的折扣率

n_dz = data_zk['店名_y'].value_counts()
n_zs = result3_data2['店名_y'].value_counts()
result4_dzspbl = pd.DataFrame({'打折商品数':n_dz,'商品总数':n_zs})
result4_dzspbl['参与打折商品比例'] = result4_dzspbl['打折商品数'] / result4_dzspbl['商品总数']
result4_dzspbl.dropna(inplace = True)
#print(result4_dzspbl.head())
# 计算出不同品牌参与打折商品比例

result4_sum = result2_data.copy()
# 筛选出品牌参加双11活动的商品总数

result4_data = pd.merge(pd.DataFrame(result4_zkld),result4_dzspbl,left_index = True, right_index = True, how = 'inner')
result4_data = pd.merge(result4_data,result4_sum,left_index = True, right_index = True, how = 'inner')
# 合并数据

result3_data2.head(20)

NameError: name 'result2_data' is not defined

In [12]:
# 用bokeh绘制散点图，x轴为参与打折商品比例，y轴为折扣力度，点的大小代表该品牌参加双11活动的商品总数

from bokeh.models.annotations import Span            # 导入Span模块
from bokeh.models.annotations import Label           # 导入Label模块
from bokeh.models.annotations import BoxAnnotation   # 导入BoxAnnotation模块


bokeh_data = result4_data[['zkl','sum','参与打折商品比例']]
bokeh_data.columns = ['zkl','amount','pre']
bokeh_data['size'] = bokeh_data['amount'] * 0.03
source = ColumnDataSource(bokeh_data)
# 创建ColumnDataSource数据

x_mean = bokeh_data['pre'].mean()
y_mean = bokeh_data['zkl'].mean()

hover = HoverTool(tooltips=[("品牌", "@index"),
                            ("折扣率", "@zkl"),
                            ("商品总数", "@amount"),
                            ("参与打折商品比例", "@pre"),
                           ])  # 设置标签显示内容
p = figure(plot_width=600, plot_height=600,
                title="各个品牌打折套路解析" , 
                tools=[hover,'box_select,reset,wheel_zoom,pan,crosshair']) 
# 构建绘图空间

p.circle_x(x = 'pre',y = 'zkl',source = source,size = 'size',
           fill_color = 'red',line_color = 'black',fill_alpha = 0.6,line_dash = [8,3])
p.ygrid.grid_line_dash = [6, 4]
p.xgrid.grid_line_dash = [6, 4]
# 散点图

x = Span(location=x_mean, dimension='height', line_color='green',line_alpha = 0.7, line_width=1.5, line_dash = [6,4])
y = Span(location=y_mean, dimension='width', line_color='green',line_alpha = 0.7, line_width=1.5, line_dash = [6,4])
p.add_layout(x)
p.add_layout(y)
# 绘制辅助线

bg1 = BoxAnnotation(bottom=y_mean, right=x_mean,fill_alpha=0.1, fill_color='olive')
label1 = Label(x=0.1, y=0.55,text="少量大打折",text_font_size="10pt" )
p.add_layout(bg1)
p.add_layout(label1)
# 绘制第一象限

bg2 = BoxAnnotation(bottom=y_mean, left=x_mean,fill_alpha=0.1, fill_color='firebrick')
label2 = Label(x=0.7, y=0.55,text="大量大打折",text_font_size="10pt" )
p.add_layout(bg2)
p.add_layout(label2)
# 绘制第二象限

bg3 = BoxAnnotation(top=y_mean, right=x_mean,fill_alpha=0.1, fill_color='firebrick')
label3 = Label(x=0.1, y=0.80,text="少量少打折",text_font_size="10pt" )
p.add_layout(bg3)
p.add_layout(label3)
# 绘制第三象限

bg4 = BoxAnnotation(top=y_mean, left=x_mean,fill_alpha=0.1, fill_color='olive')
label4 = Label(x=0.7, y=0.80,text="少量大打折",text_font_size="10pt" )
p.add_layout(bg4)
p.add_layout(label4)
# 绘制第四象限

show(p)

'''
结论：
少量少打折：包括雅诗兰黛、娇兰、兰蔻、薇姿、玉兰油等共5个品牌。
少量大打折：包括悦诗风吟、兰芝、欧珀莱等3个品牌。该类品牌的打折商品较少，但折扣力度较大。
大量小打折：包括妮维雅、美宝莲、蜜丝佛陀等3个品牌。该类型有半数以上的商品都参与了打折活动，但折扣力度并不大。
大量大打折：包括相宜本草、佰草集、自然堂等三大国产品牌。这些品牌不仅有90%以上的商品参与了折扣活动，而且折扣力度很大。
'''

NameError: name 'result4_data' is not defined